In [1]:
import tensorflow as tf

In [2]:
import glob
import collections
import numpy as np
import random
import math
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words
from nltk.stem.porter import PorterStemmer
import os

from tensorflow.contrib.tensorboard.plugins import projector


from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
PATH = os.getcwd()
LOG_DIR = PATH + '/log'
metadata = os.path.join(LOG_DIR, 'metadata.tsv')


In [4]:
metadata

'/home/parth/work_files/summer-2018/word2vect_python/log/metadata.tsv'

In [5]:
stemmer_ = PorterStemmer()

In [6]:
print(stop_words.ENGLISH_STOP_WORDS)

frozenset({'up', 'de', 'mostly', 'any', 'wherein', 'again', 'but', 'our', 'is', 'everywhere', 'thick', 'front', 'or', 'amongst', 'across', 'back', 'hundred', 'me', 'no', 'system', 'mill', 'per', 'throughout', 'at', 'whatever', 'then', 'third', 'still', 'once', 'empty', 'with', 'hasnt', 'its', 'must', 'next', 'behind', 'always', 'beforehand', 'keep', 're', 'had', 'sincere', 'being', 'latter', 'wherever', 'also', 'else', 'will', 'because', 'take', 'thereafter', 'yourselves', 'these', 'last', 'were', 'co', 'afterwards', 'how', 'part', 'often', 'rather', 'whom', 'seem', 'found', 'five', 'another', 'former', 'if', 'around', 'can', 'eleven', 'formerly', 'whereupon', 'this', 'first', 'yourself', 'four', 'such', 'of', 'much', 'my', 'three', 'be', 'detail', 'nevertheless', 'he', 'not', 'whence', 'who', 'yours', 'more', 'indeed', 'side', 'there', 'nothing', 'off', 'each', 'cant', 'somehow', 'whether', 'anyhow', 'un', 'a', 'we', 'cannot', 'down', 'by', 'yet', 'ourselves', 'would', 'cry', 'full', 

In [7]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
paths = glob.glob('MorpedTextedWWO/*.txt')

In [9]:
newFullText  = ""

In [10]:
for path in paths:
    with open(path, "r") as f:
        newFullText += f.read().split("\n")[1] 

In [11]:
newFullText = newFullText.lower()

In [12]:
vocabulary_size = 90000

In [13]:
words_all = newFullText.split(" ")

In [14]:
words_all = [wordnet_lemmatizer.lemmatize(w) for w in words_all if w not in stop_words.ENGLISH_STOP_WORDS]

In [15]:
len(set(words_all))

124951

In [16]:
len(words_all)

6758697

In [17]:
def build_dataset(words, n_words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(n_words - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        index = dictionary.get(word, 0)
        if index == 0:
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reversed_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reversed_dictionary

In [18]:
data, count, dictionary, reverse_dictionary = build_dataset(words_all, vocabulary_size)

In [19]:
data_index = 0
def createBatch(size, num_skips, skip_window):
    global data_index
    batch = np.ndarray(shape=(size), dtype=np.int32)
    labels = np.ndarray(shape=(size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span) # pylint: disable=redefined-builtin    
    
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    
    for i in range(size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
            
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

In [20]:
with open(metadata, 'w') as metadata_file:
    for i in range(vocabulary_size):
        metadata_file.write(reverse_dictionary[i] + '\n')


In [21]:
del words_all

In [22]:
del newFullText

In [23]:
batch_size = 128
embedding_size = 128
skip_window = 1
num_skips = 2
num_sampled = 64

In [24]:
valid_size = 16  # Random set of words to evaluate similarity on.
valid_window = 100  # Only pick dev samples in the head of the distribution.
valid_examples = np.random.choice(valid_window, valid_size, replace=False)

In [25]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('inputs'):
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        
        
    with tf.device('/cpu:0'):
        with tf.name_scope('embeddings'):
            embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)

        with tf.name_scope('weights'):
            nce_weights = tf.Variable(
                  tf.truncated_normal(
                  [vocabulary_size, embedding_size],
                  stddev=1.0 / math.sqrt(embedding_size)))
        with tf.name_scope('biases'):
            nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
        with tf.name_scope('loss'):
            loss = tf.reduce_mean(
                tf.nn.nce_loss(
                    weights=nce_weights,
                    biases=nce_biases,
                    labels=train_labels,
                    inputs=embed,
                    num_sampled=num_sampled,
                    num_classes=vocabulary_size))
        with tf.name_scope('optimizer'):
            optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)

        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
        similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
        init = tf.global_variables_initializer()
        
        saver = tf.train.Saver()



Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [29]:
num_steps = 100001
with tf.Session(graph=graph) as session:
    init.run()
    average_loss = 0
    for step in range(num_steps):
        batch_inputs, batch_labels = createBatch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_labels: batch_labels}
        
        _, loss_val = session.run(
            [optimizer, loss],
            feed_dict=feed_dict)
        average_loss += loss_val
        
        if step % 2000 == 0 and step > 0:
            average_loss /= 2000
            print('Average loss at step ', step, ': ', average_loss)
            average_loss = 0
        
        
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_examples[i]]
                top_k = 8
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = 'Nearest to %s:' % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = '%s %s,' % (log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()
    
    saver.save(session, os.path.join(LOG_DIR, 'model.ckpt'))

    config = projector.ProjectorConfig()

    embedding_conf = config.embeddings.add()
    embedding_conf.tensor_name = embeddings.name
    embedding_conf.metadata_path = metadata
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

    

Nearest to old: misteris, uninstructed, forwardness, presered, unpardonable, nave, transfused, manyfolde,
Nearest to hath: attrocious, crease, inficit, tassel, culling, sod, ainslie, phidias,
Nearest to mr.: unquiets, elogies, mediate, tamely, modifying, hollis, ghospell, reackoninge,
Nearest to thing: ewells, vortigern, fidentia's, boulogne, boddington, sluice, grunt, hog-grubber,
Nearest to country: schedule, odedience, cinchona, quiescit, clamav, unshipwrecked, impalement, trismegiste,
Nearest to thy: nuptialls, champaigne, snipy, tyl, sanballet, finally, wall-creeper, preare,
Nearest to tell: subjectes, wiggins, redress, lay-man, tenderhearted, blame, romantick, accrues,
Nearest to dear: agréeing, buzing, buckinghamshire, mezentius, benefyghtes, th'immortal, hyacinthus, carelesness,
Nearest to it's: accad, jav'lin, spill, cheerless, piggior, skinny, proportion, over-acted,
Nearest to young: uzzah, hue, minysshe, wheatear, animall, bottomless, illustrata, landskip-tapistry,
Nearest 

Nearest to come: dong, UNK, —, gifford, requite, set, relying, ?,
Nearest to nature: ravished, loveday, dong, cared, ross, m‘intosh, masse, wheatley,
Nearest to men: man, spooner, dong, woman, sentiment, ;, eighty-two, spirit,
Average loss at step  52000 :  7.20981882417202
Average loss at step  54000 :  6.109973956108093
Average loss at step  56000 :  6.30393492603302
Average loss at step  58000 :  6.808628868103027
Average loss at step  60000 :  7.090895910978317
Nearest to old: nave, young, uninstructed, forwardness, act, early, pretty, m‘intosh,
Nearest to hath: culling, (, carolinian, gallic, churche, glynne, perfection, innovation,
Nearest to mr.: sir, mrs., tamely, UNK, mediate, ., naval, cherish,
Nearest to thing: way, ), fraser, woman, dong, belcher, man, let,
Nearest to country: belcher, wheat-ear, agent, metcalfe, haslewood, signer, UNK, bashful,
Nearest to thy: ,, :, ;, god, becca-fica, doe, thee, wheat-ear,
Nearest to tell: say, told, ross, m‘intosh, spooner, redress, tran

In [30]:
def top_n_words(word, final_embeddings, k=10):
    word_id = dictionary[word]
    similarities = cosine_similarity([final_embeddings[word_id]], final_embeddings)[0]
    sort = np.argsort(similarities)[::-1]
    probs = []
    for x in sort[0:k]:
        probs.append((similarities[x], reverse_dictionary[x]))
    return probs

In [31]:
top_n_words("founded", final_embeddings, k=10)

[(1.0000002, 'founded'),
 (0.509346, ','),
 (0.4926116, 'harford'),
 (0.47872818, 'uninjured'),
 (0.47564012, 'porcelline'),
 (0.47105646, 'shall'),
 (0.4703773, 'vallies'),
 (0.4694081, 'tael'),
 (0.4671404, 'brav'),
 (0.46606153, 'bon-gre')]